In [1]:
import winsound
import pandas as pd 
import numpy as np
import pickle 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations
from itertools import permutations

##### UTILITIES #####

def beepStart():
    d = 150
    freq = [2500,3000,3500,4000]
    for i in freq:
        winsound.Beep(i, d)
        
def beepMid():
    d = 100
    freq = [3000,3000,3000]
    for i in freq:
        winsound.Beep(i, d)        
        
def beepEnd():
    d = 150
    freq = [4000,3600,4000,3600]
    for i in freq:
        winsound.Beep(i, d)
################

def acceptanceProbability (bestMSE, newMSE, temp , K):
        # If the new solution is better, accept it
        if (newMSE < bestMSE):
            return 1.0
        # If the new solution is worse, calculate an acceptance probability
        return np.exp( K * (newMSE - bestMSE) / temp)
    
def calcMSE(combDesc, x_train, x_val, y_train, y_val, model):
    x_train_slice = x_train.iloc[:,combDesc]
    x_val_slice = x_val.iloc[:,combDesc]
    model.fit(x_train_slice, y_train)
    y_pred = model.predict(x_val_slice)
    return mean_squared_error(y_val, y_pred)

unpick_dtTrain = pd.read_pickle("./dtTrain.pkl")
inDt = open("dtTrain.pkl","rb")
dtTrain = pickle.load(inDt)
# dtTrain.head()

In [2]:
##### SIMULATED ANNEALING #####

# 0. Preparation Data
train, val = train_test_split(dtTrain, test_size = 0.2, random_state = 45)

x_train = train.iloc[:,:-1]
x_val = val.iloc[:,:-1]
y_train = train.iloc[:, [-1]]
y_val = val.iloc[:,[-1]]

model = LinearRegression()

In [16]:
##### INTO DE SA DE LA'SOIN #####

### 3.1 Initialize solution & Best MSE (Random 6 descriptor)
r = np.random.RandomState(45)
bestSol = r.choice(x_train.shape[1],6)
bestSol = list(bestSol)
bestSol.sort()
print("Initial Best Descriptors : ", bestSol)

# MSE
bestMSE = calcMSE(bestSol, x_train, x_val, y_train, y_val, model)
print("Initial Best MSE : ", bestMSE)

### 3.2 Initialize parameters
initTemp, temp = 100, 100
tempEnd = 0.1
cooling_rate = 0.95
iteration = 20

### 3.3 Simulated Anyeong
bestList = []
sounded = True

print('Loop is Running')
beepStart()
while temp > tempEnd:
    for i in range(iteration):
        
        #Create new solution & new MSE
        newSol = np.random.choice(x_train.shape[1], 6, replace="True")
        newMSE = calcMSE(newSol, x_train, x_val, y_train, y_val, model)
        deltaMSE = newMSE - bestMSE
        K = (initTemp * np.log(0.8)) / deltaMSE

        #Calculate Acceptance
        if acceptanceProbability (bestMSE, newMSE, temp , K) > np.random.uniform(0,1):
            currMSE = newMSE
            currSol = newSol
        
        #Comparing current Solution & MSE w/ best Solution & MSE
        if currMSE < bestMSE:
            bestMSE = currMSE
            bestSol = currSol
            bestList.append([bestMSE,bestSol])
    temp *= cooling_rate
    
    if(temp < 50 and sounded):
        sounded = False
        beepMid()
        
print('Loop done!')
beepEnd()

dfBest = pd.DataFrame(bestList)
print('===========================')
print('Best Descriptors : ', dfBest.iloc[-1][1])
print('Best MSE : ', dfBest.iloc[-1][0])
dfBest

# BestDescriptors=  [31,18,54,36,8,51]
# BestMSE =  0.24519892533514573

Initial Best Descriptors :  [3, 30, 32, 35, 61, 68]
Initial Best MSE :  0.7427207240139955
Loop is Running
Loop done!
Best Descriptors :  [35  5 15 51  8 48]
Best MSE :  0.31939371851694986


,0,1
0,0.328703,"[7, 17, 51, 68, 61, 38]"
1,0.319394,"[35, 5, 15, 51, 8, 48]"
